In [169]:
class Vehicle:
    def __init__(self, name, speed):
        self.name = name
        self.speed = speed
    def move(self, distance):
        return 0

In [170]:
class Land(Vehicle): #Land inherits from Vehicle 
    def __init__(self, name, speed, interval, dur): #Moving time and Resting time
        super().__init__(name, speed) #super() in Single Inheritance
        self.interval = interval
        self.dur = dur
    def move(self, dist1): #override
        if (self.speed > 0):
            time = (dist1 / self.speed)
            round1 = int(time // self.interval)  # How many times rested
            return time + sum(self.dur[:round1]) + (round1 - len(self.dur)) * (round1 >= len(self.dur)) * self.dur[-1]
        else:
            return float('inf')
        #self.speed * self.interval # Covered Distance
    def __repr__(self):
        return "Land({})".format(self.name)

In [171]:
class Air(Vehicle): #Air inherits from Vehicle 
    def __init__(self, name, speed, redu): # redu = [(0, 1000), (3, 5000), ...] or [(6, 0)] or [(1, 1000, 0)]
        super().__init__(name, speed) #super() in Single Inheritance
        self.redu = redu 
    def move(self, dist): #override
        if(self.speed > 0):
            if len(self.redu) == 1: # If every 1000 m has 1% reduction or every distance has 6% reduction
                if self.redu[0][1] == 0: # Every distance has 6%
                    required = dist * (1 - self.redu[0][0] / 100)
                else: # 1000m has 1%
                    required = (dist // self.redu[0][1]) * (1 - self.redu[0][0] / 100) * self.redu[0][1] + dist % self.redu[0][1]
            else:
                for i in self.redu: # Finding the interval
                    if dist <= i[1]:
                        required = dist * (1 - i[0] / 100) 
                        break
                else: # more than last one.
                    required = dist * (1 - self.redu[-1][0] / 100)
            return required / self.speed
        else:
            return float('inf')
    def __repr__(self):
        return "Air({})".format(self.name)

In [172]:
class Race(Air, Land):
    def __init__(self, ty): # type of Race (Land, Air, 0) 0 - Mixed | Distance 
        self.ty = ty
        self.trans = [] # participater
    def add_transport(self, trans):
        if self.ty == 0 or self.ty == type(trans):
            self.trans.append(trans)
        else:
            print("Wrong race type")
    def winner(self, dist):
        if dist <= 0:
            return "error"
        else:
            return sorted([(i, i.move(dist)) for i in self.trans if i.move(dist) != float('inf')], key = lambda x: x[1])
    def __repr__(self):
        return "Race({})".format(self.trans)

In [173]:
#Create Vehicles
camel1 = Land('bactrian_camel', 10, 30, [5,8])
camel2 = Land('fast_camel', 40, 10, [5, 6.5, 8])
centaur = Land('centaur', 15, 8, [2])
boot = Land('boot', 6, 60, [10, 5])
carpet = Air('carpet', 10, [(0, 1000), (3, 5000), (10, 10000), (5, 10000)])
stupa = Air('stupa', 8, [(6, 0)])
broom = Air('broom', 20, [(1, 1000, 0)])

In [174]:
camel1

Land(bactrian_camel)

In [175]:
broom

Air(broom)

In [176]:
#Create races
landrace = Race(Land)
airrace = Race(Air)
mixedrace = Race(0)

In [177]:
# Add transports to races
landrace.add_transport(camel1)
landrace.add_transport(camel2)
landrace.add_transport(centaur)
landrace.add_transport(boot)

airrace.add_transport(carpet)
airrace.add_transport(stupa)
airrace.add_transport(broom)

mixedrace.add_transport(camel1)
mixedrace.add_transport(camel2)
mixedrace.add_transport(centaur)
mixedrace.add_transport(boot)
mixedrace.add_transport(carpet)
mixedrace.add_transport(stupa)
mixedrace.add_transport(broom)

In [178]:
#Show transports in race
landrace

Race([Land(bactrian_camel), Land(fast_camel), Land(centaur), Land(boot)])

In [179]:
#Show transports in race
mixedrace

Race([Land(bactrian_camel), Land(fast_camel), Land(centaur), Land(boot), Air(carpet), Air(stupa), Air(broom)])

In [180]:
#Show transports in race
airrace

Race([Air(carpet), Air(stupa), Air(broom)])

In [181]:
# Show land type can't enter in air race
airrace.add_transport(camel1)

Wrong race type


In [182]:
# Vice versa
landrace.add_transport(broom)

Wrong race type


In [183]:
#Time of bactrian_camel
camel1.move(1000)

121.0

In [184]:
# name, time-> sorted by time in asc
airrace.winner(1000) 

[(Air(broom), 49.5), (Air(carpet), 100.0), (Air(stupa), 117.5)]

In [185]:
# name, time-> sorted by time in asc
landrace.winner(1000)

[(Land(fast_camel), 36.5),
 (Land(centaur), 82.66666666666667),
 (Land(bactrian_camel), 121.0),
 (Land(boot), 181.66666666666666)]

In [186]:
#name, time-> sorted by time in asc
mixedrace.winner(1000)

[(Land(fast_camel), 36.5),
 (Air(broom), 49.5),
 (Land(centaur), 82.66666666666667),
 (Air(carpet), 100.0),
 (Air(stupa), 117.5),
 (Land(bactrian_camel), 121.0),
 (Land(boot), 181.66666666666666)]

In [187]:
#Lets create new vehicle and start the race
wings = Air('wings', 200, [(3, 1000, 0)])
airrace.add_transport(wings)
mixedrace.add_transport(wings)

In [188]:
#New winner in air race
airrace.winner(1000)

[(Air(wings), 4.85),
 (Air(broom), 49.5),
 (Air(carpet), 100.0),
 (Air(stupa), 117.5)]

In [189]:
#wings can't participate in land 
landrace.add_transport(wings)

Wrong race type


In [190]:
#New winner in mixed race
mixedrace.winner(1000)

[(Air(wings), 4.85),
 (Land(fast_camel), 36.5),
 (Air(broom), 49.5),
 (Land(centaur), 82.66666666666667),
 (Air(carpet), 100.0),
 (Air(stupa), 117.5),
 (Land(bactrian_camel), 121.0),
 (Land(boot), 181.66666666666666)]

In [191]:
#Speed can't be negative
fish = Air('fly', -5, [(3, 1000, 0)])
airrace.add_transport(fish)
fish.move(1000)

inf

In [192]:
airrace.winner(1000)

[(Air(wings), 4.85),
 (Air(broom), 49.5),
 (Air(carpet), 100.0),
 (Air(stupa), 117.5)]